In [1]:
from tkinter import messagebox
import openpyxl
from pathlib import Path
from datetime import date, datetime
from tkinter import *
from tkcalendar import DateEntry
from tkinter import messagebox, Entry, Button, Text, END, Tk, Toplevel
import os
from tkinter import filedialog
from reportlab.pdfgen import canvas


                                # Get the current working directory
current_directory = os.getcwd()

                                             # Specify the Excel file name
excel_file_name = "registration_data.xlsx"

                                                                                   # Build the full path to the Excel file
excel_file_path = os.path.join(current_directory, excel_file_name)

                                                 # Initialize medicine_entries globally
medicine_entries = []

                                    # Define input_name1 in the global scope
input_name1 = None
input_country = None
input_phone = None
input_br = None
input_addre= None
v = None
gen = None
hos = None
sheet = None

try:
    workbook = openpyxl.load_workbook(excel_file_path)
    sheet = workbook.active
except FileNotFoundError:
    try:
        # Attempt to create the workbook and sheet
        workbook = openpyxl.Workbook()
        sheet = workbook.active

        # Check if the first row already contains column names
        if sheet.cell(row=1, column=1).value is None:
            # If not, add column names
            sheet['A1'] = "DATE"
            sheet['B1'] = "NAME"
            sheet['C1'] = "AGE"
            sheet['D1'] = "HOSPITAL"
            sheet['E1'] = "GENDER"
            sheet['F1'] = "COUNTRY"
            sheet['G1'] = "PHONE NO."
            sheet['H1'] = "ADDRESS"
            sheet['I1'] = "MEDICINE"
            sheet['J1'] = "QUANTITY"
            sheet['K1'] ="PER PIECE."
            sheet['l1'] = "TOTAL BILL"
            

        # Save the workbook with the specified file name
        workbook.save(excel_file_path)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while creating the Excel file: {e}")

def clear_fields():
    input_name1.delete(0, END)
    input_country.delete(0, END)
    input_phone.delete(0, END)
    input_br.delete(1.0, END)
    input_addre.delete(0, END)
    v.set(0)

def print_report():
    try:
        # Get the current date and time for the file name
        current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")

        # Ask the user to choose a location to save the PDF
        file_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")], initialfile=f"report_{current_datetime}")

        # Create a PDF document
        c = canvas.Canvas(file_path)

        # Add content to the PDF
        c.drawString(72, 800, "__________CUSTOMER_____DETAILS__________")

        # Add information from input_br
        input_br_text = input_br.get("1.0", END).strip()
        c.drawString(72, 780, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        c.drawString(72, 760, f"NAME: {input_name1.get()}")
        c.drawString(72, 740, f"AGE: {v.get()}")
        c.drawString(72, 720, f"GENDER : {gen.get()}")
        c.drawString(72, 700, f"HOSPITAL : {hos.get()}")
        c.drawString(72, 680, f"COUNTRY : {input_country.get()}")
        c.drawString(72, 660, f"PHONE NUMBER : {input_phone.get()}")
        c.drawString(72, 640, f"ADDRESS : {input_addre.get()}")

        # Add information from combine_med, combine_quantity, combine_perprice, and combine_price
        c.drawString(72, 620, "___MEDICINE___DETAILS___")
        c.drawString(72, 600, "Medicine          Quantity           Per Price           Total Price")

        combine_med_text = combine_med.get("1.0", END).strip()
        combine_quantity_text = combine_quantity.get("1.0", END).strip()
        combine_perprice_text = combine_perprice.get("1.0", END).strip()
        combine_price_text = combine_price.get("1.0", END).strip()

        # Split the text into lines
        med_lines = combine_med_text.split("\n")
        quantity_lines = combine_quantity_text.split("\n")
        perprice_lines = combine_perprice_text.split("\n")
        price_lines = combine_price_text.split("\n")

        # Iterate over the lines and add them to the PDF
        for i in range(max(len(med_lines), len(quantity_lines), len(perprice_lines), len(price_lines))):
            med_value = med_lines[i] if i < len(med_lines) else ""
            quantity_value = quantity_lines[i] if i < len(quantity_lines) else ""
            perprice_value = perprice_lines[i] if i < len(perprice_lines) else ""
            price_value = price_lines[i] if i < len(price_lines) else ""

            line_text = f"{med_value:<20}{quantity_value:<20}{perprice_value:<20}{price_value}"
            c.drawString(72, 580 - i * 20, line_text)

        # Add grand total information
        grand_total_text = grandtotal_price.get("1.0", END).strip()
        c.drawString(72, 560 - i * 20, f"Grand Total: {grand_total_text}")

        # Save and close the PDF
        c.save()

        # Display success message
        messagebox.showinfo("Success", f"Report saved as PDF: {file_path}")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while creating the PDF: {e}")



submit_clicked = False

def handle_login():
    global submit_clicked
    current_date = datetime.now().strftime("%Y-%m-%d")
    input_date.delete(0, END)
    input_date.insert(0, current_date)

    if len(input_name1.get()) == 0 or not input_name1.get().isalpha():
        messagebox.showerror('REGISTRATION FORM', 'INVALID NAME')
        return

    if hos.get() == "" or gen.get() == "":
        messagebox.showerror('REGISTRATION FORM', 'Please select hospital and gender')
        return

    if len(input_country.get()) == 0 or not input_country.get().isalpha():
        messagebox.showerror('REGISTRATION FORM', 'Invalid Country')
        return

    if len(input_phone.get()) == 0 or not (input_phone.get().isdigit() and len(input_phone.get()) == 10):
        messagebox.showerror('REGISTRATION FORM', 'PHONE NUMBER OF 10 DIGITS')
        return
    if len(input_addre.get()) == 0:
        messagebox.showerror('REGISTRATION FORM', 'Invalid Address')
        return

    try:
        # Use a fixed row number (e.g., 2) for sequential storage
        current_row = 2

        # Find the first empty row
        while sheet.cell(row=current_row, column=1).value is not None:
            current_row += 1

        # Display information in the Text widget
        inf1 = f"NAME         : {input_name1.get()}"
        inf2 = f"AGE           : {v.get()}"
        inf3 = f"GENDER      : {('Male' if gen.get() == 0 else 'Female')}"
        inf4 = f"HOSPITAL    : {('GOVT' if hos.get() == 0 else 'PVT')}"
        inf5 = f"COUNTRY    : {input_country.get()}"
        inf6 = f"PHONE NUMBER : {input_phone.get()}"
        inf7 = f"DATE         : {current_date}"
        inf8 = f"ADDRESS         : {input_addre.get()}"
        info = f"{inf7}\n{inf1}\n{inf2}\n{inf3}\n{inf4}\n{inf5}\n{inf6}\n{inf8}"
        input_br.delete("1.0", END)
        input_br.insert(END, info)

        # Set submit_clicked to True
        submit_clicked = True

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")


        
        
input_quantity = None
input_price = None
inputtotal_price = None
input_med1 = None
combine_med = None
combine_quantity = None
combine_perprice = None
combine_price = None
grandtotal_price = None


def calculate_total_price():
    try:
        global inputtotal_price  # Declare as global to modify the global variable
        
        # Get the values from input_quantity and input_price
        quantity = int(input_quantity.get())
        price = float(input_price.get())

        # Check if quantity and price are greater than 0
        if quantity > 0 and price > 0:
            # Calculate the total price
            total_price = quantity * price

            # Display the total price in the inputtotal_price Entry widget
            inputtotal_price.delete(0, END)  # Clear any previous value
            inputtotal_price.insert(0, total_price)
        else:
            messagebox.showerror('Error', 'Quantity and price must be greater than 0.')
    except ValueError:
        messagebox.showerror('Error', 'Invalid input. Please enter valid numeric values for quantity and price.')

def add_to_combine_med():
    # Retrieve values from input_med1, input_quantity, input_price, and inputtotal_price
    med_value = input_med1.get()
    quantity_value = input_quantity.get()
    price_value = input_price.get()
    total_price_value = inputtotal_price.get()

    # Set the length to 0
    input_med1.delete(0, END)
    input_quantity.delete(0, END)
    input_price.delete(0, END)
    inputtotal_price.delete(0, END)

    # Check conditions before adding to combine_med
    if len(med_value) > 0:
        try:
            # Check if quantity and price are valid integers greater than 0
            if int(quantity_value) > 0 and int(price_value) > 0:
                # Append the values to combine_med Text widget
                combine_med.insert(END, f"{med_value  }\n")
                combine_quantity.insert(END, f" {quantity_value}\n")
                combine_perprice.insert(END, f" {price_value}\n")
                    
                # Append total_price_value to combine_price Text widget
                combine_price.insert(END, f"{total_price_value}\n")

            else:
                messagebox.showerror('Error', 'Quantity and price must be valid integers greater than 0.')
        except ValueError:
            messagebox.showerror('Error', 'Quantity and price must be valid integers greater than 0.')
    else:
        messagebox.showerror('Error', 'Medicine name cannot be empty.')
  


def clear_entries():
    # Clear the content of grandtotal_price and combine_med
    grandtotal_price.delete(1.0, END)
    combine_med.delete(1.0, END)
    combine_quantity.delete(1.0,END)
    combine_perprice.delete(1.0,END)
    combine_price.delete(1.0,END)
    
def submit_data():
    global grandtotal_price, combine_med, combine_quantity, combine_perprice, combine_price

    try:
        # Open the Excel file for editing
        workbook = openpyxl.load_workbook(excel_file_path)
        sheet = workbook.active

        # Find the first empty row
        current_row = 2
        while sheet.cell(row=current_row, column=1).value is not None:
            current_row += 1

        # Store the data in the Excel sheet
        sheet.cell(row=current_row, column=1, value=date.today().strftime("%Y-%m-%d"))
        sheet.cell(row=current_row, column=2, value=input_name1.get())

        # Convert age to integer with error handling
        try:
            sheet.cell(row=current_row, column=3, value=int(v.get()))
        except ValueError:
            messagebox.showerror('Error', 'Invalid age. Please enter a valid integer.')
            return

        gender_value = 'Male' if gen.get() == 0 else 'Female'
        sheet.cell(row=current_row, column=4, value='GOVT' if hos.get() == 0 else 'PVT')
        sheet.cell(row=current_row, column=5, value=gender_value)
        sheet.cell(row=current_row, column=6, value=input_country.get())
        sheet.cell(row=current_row, column=7, value=input_phone.get())
        sheet.cell(row=current_row, column=8, value=input_addre.get())

        # Store the medicine-related data in the same row
        sheet.cell(row=current_row, column=9, value=combine_med.get("1.0", END).strip())
        sheet.cell(row=current_row, column=10, value=combine_quantity.get("1.0", END).strip())
        sheet.cell(row=current_row, column=11, value=combine_perprice.get("1.0", END).strip())
        sheet.cell(row=current_row, column=12, value=grandtotal_price.get("1.0", END).strip())
        


        # Save the changes to the Excel file
        workbook.save(excel_file_path)

        # Display success message
        messagebox.showinfo("Success", "Data saved to Excel file!")

        # Clear the content of grandtotal_price, combine_med, and combine_price
        grandtotal_price.delete(1.0, END)
        combine_med.delete(1.0, END)
        combine_quantity.delete(1.0, END)
        combine_perprice.delete(1.0, END)
        combine_price.delete(1.0, END)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

        
def calculate_grand_total():
    try:
        # Get all the lines from combine_price
        lines = combine_price.get("1.0", END).split("\n")

        # Remove the last empty line
        lines = lines[:-1]

        # Extract the total price values, filtering out empty strings
        total_prices = [float(line.strip()) for line in lines if line.strip()]

        # Calculate the grand total
        grand_total_value = sum(total_prices)

        # Display the grand total in the grandtotal_price Text widget
        grandtotal_price.delete(1.0, END)  # Clear any previous value
        grandtotal_price.insert(END, grand_total_value)

    except ValueError:
        messagebox.showerror('Error', 'Invalid input in medicine prices.')



def create_new_frame():
    global input_quantity, input_price, inputtotal_price, input_med1, combine_med, combine_quantity, combine_perprice, combine_price, grandtotal_price

    # Check if the submit button has been clicked
    if not submit_clicked:
        messagebox.showinfo("Information", "Please submit the form first.")
        return
    
    fr.withdraw()
    new_frame = Toplevel(fr)
    new_frame.title("Medicine Entry")
    new_frame.geometry("900x650")
    
    
    titlelabel = Label(new_frame, text="MEDICINE DETAILS", font=('Times New Roman', 20), bg='black', fg='white', pady=10)
    titlelabel.pack(side=TOP, fill=X)

    med_label = Label(new_frame, text="MEDICINE", font=('Times New Roman', 20), bd=5, relief="ridge")
    med_label.place(x=10, y=90)
    
    input_med1 = Entry(new_frame, width=18, font=('Times New Roman', 20), bd=3, relief="solid")
    input_med1.place(x=10, y=150)
    
    quantity_label = Label(new_frame, text="QUANTITY", font=('Times New Roman', 20), bd=5, relief="ridge")
    quantity_label.place(x=280, y=90)
    
    input_quantity = Entry(new_frame, width=10, font=('Times New Roman', 20), bd=3, relief="solid")
    input_quantity.place(x=280, y=150)
    
    price_label = Label(new_frame, text="PER PRICE", font=('Times New Roman', 20), bd=5, relief="ridge")
    price_label.place(x=450, y=90)
    
    input_price = Entry(new_frame, width=10, font=('Times New Roman', 20), bd=3, relief="solid")
    input_price.place(x=450, y=150)

    totalprice_button = Button(new_frame, text="TOTAL PRICE", bg='orange', fg='black', font=('Times New Roman', 20), bd=5, relief="ridge", command=calculate_total_price)
    totalprice_button.place(x=610, y=80)

    inputtotal_price = Entry(new_frame, width=14, font=('Times New Roman', 20), bd=3, relief="solid")
    inputtotal_price.place(x=610, y=150)
    
    insertmed = Button(new_frame, text='ADD', bg='sky blue', fg='black', width=4, height=2, command=add_to_combine_med, font=('Times New Roman', 20))
    insertmed.place(x=820, y=100)
    
    combine_med = Text(new_frame, width=23, height=10, bd=3, font=('Times New Roman', 16))
    combine_med.place(x=10, y=200)
    
    combine_quantity = Text(new_frame, width=13, height=10, bd=3, font=('Times New Roman', 16))
    combine_quantity.place(x=280, y=200)
    
    combine_perprice = Text(new_frame, width=13, height=10, bd=3, font=('Times New Roman', 16))
    combine_perprice.place(x=450, y=200)
    
    combine_price = Text(new_frame, width=17, height=10, bd=3, font=('Times New Roman', 16))
    combine_price.place(x=610, y=200)
    
    grand_total = Button(new_frame, text='TOTAL', bg='yellow', fg='black', width=7, height=1, command=calculate_grand_total, font=('Times New Roman', 20))
    grand_total.place(x=470, y=440)

    grandtotal_price = Text(new_frame, width=17, height=2, bd=3, font=('Times New Roman', 16))
    grandtotal_price.place(x=610, y=440)
    
    # Add an Edit button
    clear_button = Button(new_frame, text='CLEAR', bg='orange', fg='black', width=7, height=1, font=('Times New Roman', 20), command=clear_entries)
    clear_button.place(x=10, y=500)
    
    submit_button = Button(new_frame, text='SUBMIT', bg='green', fg='black', width=7, height=1, font=('Times New Roman', 20), command=submit_data)
    submit_button.place(x=200, y=500)
    
    back_button = Button(new_frame, text='BACK', bg='green', fg='black',command=lambda: show_main_frame(new_frame),font=('Times New Roman', 20), width=7, height=1, bd=3, relief="ridge")
    back_button.place(x=380, y=500)

    exit_button = Button(new_frame, text='EXIT', bg='red', fg='black', command=new_frame.destroy,font=('Times New Roman', 20), width=7, height=1, bd=3, relief="ridge")
    exit_button.place(x=540, y=500)
    
    print_button = Button(new_frame, text='PRINT', bg='cyan', fg='black', width=7, height=1, command=print_report, font=('Times New Roman', 20))
    print_button.place(x=700, y=500)

    
    
def show_main_frame(frame):
    frame.destroy()
    fr.deiconify()

fr = Tk()
fr.title("MEDICAL STORE MANAGEMENT SYSTEM")
fr.geometry("900x650")

# Added label on top of the window
title_label = Label(fr, text="MEDICAL STORE MANAGEMENT SYSTEM", font=('Times New Roman', 20), bg='black', fg='white', pady=10)
title_label.pack(side=TOP, fill=X)

date_label = Label(fr, text="DATE           ", font=('Times New Roman', 20), bd=5, relief="ridge")
date_label.place(x=10, y=75)

input_date = DateEntry(fr, width=20, font=('Times New Roman', 20), bd=3, relief="solid")
input_date.place(x=230, y=75)

name_label = Label(fr, text="NAME          ", font=('Times New Roman', 20), bd=5, relief="ridge")
name_label.place(x=10, y=125)

input_name1 = Entry(fr, width=21, font=('Times New Roman', 20), bd=3, relief="solid")
input_name1.place(x=230, y=125)

age = Label(fr, text="AGE             ", font=('Times New Roman', 20), bd=5, relief="ridge")
age.place(x=10, y=175)

v = IntVar()
scale = Scale(fr, variable=v, from_=0, to=100, orient=HORIZONTAL, font=('Times New Roman', 13), length=300, tickinterval=100)
scale.place(x=230, y=164)

gender1 = Label(fr, text="GENDER     ", font=('Times New Roman', 20), bd=5, relief="ridge")
gender1.place(x=10, y=225)

gen = IntVar()
male_radio = Radiobutton(fr, text="Male", padx=5, variable=gen, value=0, font=('Times New Roman', 20))
male_radio.place(x=230, y=225)
female_radio = Radiobutton(fr, text="Female", padx=20, variable=gen, value=1, font=('Times New Roman', 20))
female_radio.place(x=390, y=225)

hospital = Label(fr, text="HOSPITAL   ", font=('Times New Roman', 20), bd=5, relief="ridge")
hospital.place(x=10, y=275)

hos = IntVar()
govt = Checkbutton(fr, text="GOVT", padx=5, variable=hos, onvalue=0, offvalue=1, font=('Times New Roman', 20))
govt.place(x=230, y=275)
pvt = Checkbutton(fr, text="PVT", padx=20, variable=hos, onvalue=1, offvalue=0, font=('Times New Roman', 20))
pvt.place(x=390, y=275)

country = Label(fr, text="COUNTRY   ", font=('Times New Roman', 20), bd=5, relief="ridge")
country.place(x=10, y=325)

input_country = Entry(fr, width=21, font=('Times New Roman', 20), bd=3, relief="solid")
input_country.place(x=230, y=325)

phone = Label(fr, text="PHONE NO. ", font=('Times New Roman', 20), bd=5, relief="ridge")
phone.place(x=10, y=375)

input_phone = Entry(fr, width=21, font=('Times New Roman', 20), bd=3, relief="solid")
input_phone.place(x=230, y=375)

input_br = Text(fr, width=29, height=14.50, font=('Times New Roman', 16))
input_br.place(x=570, y=80)



name_addre = Label(fr, text="ADDRESS          ", font=('Times New Roman', 20), bd=5, relief="ridge")
name_addre.place(x=10, y=425)

input_addre = Entry(fr, width=21, font=('Times New Roman', 20), bd=3, relief="solid")
input_addre.place(x=230, y=425)


submit_button = Button(fr, text='SUBMIT', bg='green', fg='black', width=13, height=1, command=handle_login, font=('Times New Roman', 20))
submit_button.place(x=10, y=485)

clear_button = Button(fr, text='CLEAR', bg='red', fg='black', width=13, height=1, command=clear_fields, font=('Times New Roman', 20))
clear_button.place(x=230, y=485)

medicine_frame_button = Button(fr, text='MEDICINE', bg='yellow', fg='black', width=13, height=1, command=create_new_frame, font=('Times New Roman', 20))
medicine_frame_button.place(x=450, y=485)

exit_button1 = Button(fr, text='EXIT', bg='red', fg='black', command=fr.destroy,font=('Times New Roman', 20), width=13, height=1, bd=3, relief="ridge")
exit_button1.place(x=670, y=485)

fr.mainloop()
